In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re
import math
usr = "kdg5188@sogang.ac.kr"
pwd = "ehdrms3479@"



In [2]:
div=['IT/인터넷',
'경영/기획/컨설팅',
'교육',
'금융/재무','디자인',
'마케팅/시장조사',
'미디어/홍보',
'법률/법무',
'생산/제조',
'생산관리/품질관리',
'서비스/고객지원',
'엔지니어링',
'연구개발',
'영업/제휴',
'유통/무역',
'의약',
'인사/총무',
'전문직',
'특수계층/공공','기타']
list_div = []
list_date =[]
list_stars = []
list_summery = []
list_merit = []
list_disadvantages = []
list_managers =[]
list_name=[]

In [3]:



# step3.크롬드라이버 실행 및 잡플래닛 로그인
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome('chromedriver.exe', options=options)
driver.get("https://www.jobplanet.co.kr") #https://www.jobplanet.co.kr/companies?&
driver.maximize_window()

time.sleep(5)
login=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > div.right_box > div.login_before.show > a.btn_txt.login > span")
login.click()

login_id = driver.find_element_by_css_selector("input#user_email")
login_id.send_keys(usr)

login_pwd = driver.find_element_by_css_selector("input#user_password")

login_pwd.send_keys(pwd)

login_id.send_keys(Keys.RETURN)
time.sleep(5)


rank=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > nav > ul > li.companies.btn_global_header > a")
rank.click()
time.sleep(3)

rank2=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > div.global_header_modal.global_header_open > div.header_modal_wrap > div > div.header_modal_left > div.modal_left_top > div > a > span")
rank2.click()
time.sleep(5)
# :nth-child 안 숫자 조정해서 산업군 별로
driver.find_element_by_css_selector("#navIndustrySide > div > div.side_body > ul > li:nth-child(3) > a > span").click()
# 산업군 선택 페이지 저장
currnet_url = driver.current_url



In [ ]:
for i in range(3,5): #range(1,11)
    driver.find_element_by_css_selector("#listCompanies > div > div.section_group > section:nth-child({}) > div > div > dl.content_col2_3.cominfo > dt > a".format(i)).click()
    #회사 페이지 들어가기
    print(i,"산업")
    try:
        WebDriverWait(driver, 45).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.txt1")))
    except:
        driver.refresh()
        time.sleep(5)
        pass
    
    #프리미엄 구매 버튼 체크
    try:
        premium = driver.find_element_by_css_selector('#premiumReviewChart > div > div.layer_popup_box.layer_popup_box_on > div.layer_popup.jply_modal.premium_review_inform > div > div.premium_modal_header > button')
        premium.click()
    except:
        pass
    #회사 이름 저장
    name=driver.find_element_by_css_selector("body > div.body_wrap > div.cmp_hd > div.new_top_bnr > div > div.top_bnr_wrap > div > div > div.company_info_sec > div.company_info_box > div.company_name > h1 > a").text
    page= math.ceil(int(driver.find_element_by_css_selector("#viewCompaniesMenu > ul > li.viewReviews > a > span").text)/5)
    
    for k in range(1,page+1): #페이지 수대로 for문
        # css_selector가 span.txt1인 element가 로딩될 때 까지 10초 대기
        try:
            WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.txt1")))
            print(k,"page element로딩완료")
        except TimeoutException:
        # 실패 시에는 에러메시지로 Time Out 출력
            print('Time Out')
            pass        
    
        #직무, 근속여부, 일시
        count=0
        list_user_info=[j.text for j in driver.find_elements_by_css_selector("span.txt1")]
        for x in list_user_info:
            if re.match("\d",x):
                count+=1
                list_date.append(x)
            elif x in div:
                list_div.append(x)

        #별점
        stars = driver.find_elements_by_css_selector("div.star_score")
        for j in stars:
            a = j.get_attribute('style')
            if a[7:9] == '20':
                list_stars.append("1점")
            elif a[7:9] == '40':
                list_stars.append("2점")
            elif a[7:9] == '60':
                list_stars.append("3점")
            elif a[7:9] == '80':
                list_stars.append("4점")
            else:
                list_stars.append("5점")
        #요약 정보
        list_summery+=[j.text for j in driver.find_elements_by_css_selector("h2.us_label")]
        #장점, 단점, 경영진에게 바라는 점

        list_review = [j.text for j in driver.find_elements_by_css_selector("dd.df1")]


        for j in range(count):      #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
            a = list_review[3*j]
            list_merit.append(a)
        
            b = list_review[3*j+1]
            list_disadvantages.append(b)

            c = list_review[3*j+2]
            list_managers.append(c)
            list_name.append(name)

        # 다음 페이지 클릭 후 for문 진행, 끝 페이지에서 다음 페이지 클릭 안되는 것 대비해서 예외처리 구문 추가
        try:
            driver.find_element_by_css_selector("#viewReviewsList > div > div > div > div.pg_bottom.um_paginnation > article > a.btn_pgnext").click()
            time.sleep(15)
            #print(k+1,"페이지넘어감")
                
        except:
            print("다음 페이지 오류")
            
    driver.get(currnet_url)        
    time.sleep(15)
    


3 산업


In [13]:
total_data = pd.DataFrame()
total_data['회사'] = pd.Series(list_name)
total_data['날짜'] = pd.Series(list_date)
total_data['직무'] = pd.Series(list_div)
total_data['별점'] = pd.Series(list_stars)
total_data['요약'] = pd.Series(list_summery)
total_data['장점'] = pd.Series(list_merit)
total_data['단점'] = pd.Series(list_disadvantages)
total_data['경영진에게 바라는 점'] = pd.Series(list_managers)

# step9.엑셀 형태로 저장하기
total_data.to_excel("의료3~4 리뷰 정리.xls" ,index=True)

# step10.크롬 드라이버 종료
driver.close()

In [8]:
[k for k in range(1,10)]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
list_disadvantages[-1]

'업무 강도'

In [7]:
page

329

In [18]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get("https://www.jobplanet.co.kr")
time.sleep(5)
login=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > div.right_box > div.login_before.show > a.btn_txt.login > span")
login.click()

login_id = driver.find_element_by_css_selector("input#user_email")
login_id.send_keys(usr)

login_pwd = driver.find_element_by_css_selector("input#user_password")

login_pwd.send_keys(pwd)

login_id.send_keys(Keys.RETURN)
time.sleep(5)


rank=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > nav > ul > li.companies.btn_global_header > a")
rank.click()
time.sleep(3)

rank2=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > div.global_header_modal.global_header_open > div.header_modal_wrap > div > div.header_modal_left > div.modal_left_top > div > a > span")
rank2.click()
time.sleep(5)
# :nth-child 안 숫자 조정해서 산업군 별로
driver.find_element_by_css_selector("#navIndustrySide > div > div.side_body > ul > li:nth-child(2) > a > span").click()
# 산업군 선택 페이지 저장
currnet_url = driver.current_url


In [ ]:
user_info = driver.find_elements_by_css_selector("span.txt1")
count = int(len(user_info)/4)
        #직무, 근속여부, 일시
list_user_info = []

for j in user_info:
    list_user_info.append(j.text)

In [17]:
list_div

['생산/제조', '생산/제조']

In [15]:
count=0
list_user_info=[j.text for j in driver.find_elements_by_css_selector("span.txt1")]
for x in list_user_info:
    if re.match("\d",x):
        count+=1
        list_date.append(x)
    elif x in div:
        list_div.append(x)

        #별점
stars = driver.find_elements_by_css_selector("div.star_score")
for j in stars:
    a = j.get_attribute('style')
    if a[7:9] == '20':
        list_stars.append("1점")
    elif a[7:9] == '40':
        list_stars.append("2점")
    elif a[7:9] == '60':
        list_stars.append("3점")
    elif a[7:9] == '80':
        list_stars.append("4점")
    else:
        list_stars.append("5점")
        #요약 정보
list_summery+=[j.text for j in driver.find_elements_by_css_selector("h2.us_label")]
        #장점, 단점, 경영진에게 바라는 점

list_review = [j.text for j in driver.find_elements_by_css_selector("dd.df1")]


for j in range(count):      #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
    a = list_review[3*j]
    list_merit.append(a)
        
    b = list_review[3*j+1]
    list_disadvantages.append(b)

    c = list_review[3*j+2]
    list_managers.append(c)
    list_name.append(name)

In [32]:
int(int(driver.find_element_by_css_selector("#viewCompaniesMenu > ul > li.viewReviews > a > span").text)/5+1)

1317